In [1]:
import sys
import os
import json
import glob
import shutil
from math import dist
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.font_manager import FontProperties
import cv2
try:
    from mmdet.apis import inference_detector, init_detector
    has_mmdet = True
except (ImportError, ModuleNotFoundError):
    has_mmdet = False
    
    
from mmpose.apis import inference_topdown
from mmpose.apis import init_model as init_pose_estimator
from mmpose.evaluation.functional import nms
from mmpose.registry import VISUALIZERS
from mmpose.structures import merge_data_samples, split_instances
from mmpose.utils import adapt_mmdet_pipeline
sys.path.append(os.path.abspath("../code_base"))
from prediction import pred_csv
from _info_ import ear_types, degrees
# from ..code_base.prediction import pred_csv
# from ..code_base._info_ import ear_types, degrees

In [10]:
class occlusion_pred_csv(pred_csv):
    def __init__(self):
        super().__init__("MAT_inpainting")
        self.occluded_result = "../occluded_result"
        self.occluded_imgs_csv = "../occluded_imgs_csv"
        
    def generate(self, has_detector = True):
        if has_detector == True:
            detector = init_detector(self.det_cfg, self.det_ckpt, device="cuda:0")
            detector.cfg = adapt_mmdet_pipeline(detector.cfg)
        else:
            detector = None
    
        for ear_type in ear_types:
            names = os.listdir(os.path.join("..", self.data_folder, ear_type, "model_save"))
            for name in names:
                rtmpose_cfg = os.path.join(self.kpt_cfg.format(et = ear_type))
                rtmpose_ckp = glob.glob(os.path.join("..", self.data_folder, ear_type, "model_save", name,"best*.pth"))[0]

                pose_estimator = init_pose_estimator(
                rtmpose_cfg,
                rtmpose_ckp,
                device="cuda:0",
                cfg_options=dict(
                    model=dict(test_cfg=dict(output_heatmaps=False))))
                for deg in degrees:
                    indexes = os.listdir(os.path.join(self.occluded_imgs_csv, ear_type, name, deg))
                    for index in indexes:
                        imgs = os.listdir(os.path.join(self.occluded_imgs_csv, ear_type, name, deg, index, "img"))
                        df = pd.DataFrame()
                        total_count = 0
                        for img in imgs:
                            image = cv2.imread(os.path.join(self.occluded_imgs_csv, ear_type, name, deg, index, "img", img))
                            kpts = self.process_one_image(image, detector, pose_estimator)
                            kpts_pd = {}
                            for i in range(0, len(kpts)):
                                x, y  = kpts[i]
                                kpts_pd[i] = str(x)+str(",")+str(y)
                            ser = pd.DataFrame(data=kpts_pd, index = [total_count])
                            df = pd.concat([df, ser])
                            total_count += 1
                        occlusion_pred = os.path.join(self.occluded_result, ear_type, name, "pred", deg, index)

                        if not os.path.isdir(occlusion_pred):
                            os.makedirs(occlusion_pred)
                        df.to_csv(os.path.join(occlusion_pred, "pred.csv"))
                
                

In [11]:
A = occlusion_pred_csv()
A.generate()

Loads checkpoint by local backend from path: ../mmdetection/work_dirs/rtmdet_nano_320-8xb32_coco-ear/epoch_120.pth
Loads checkpoint by local backend from path: ..\MAT_inpainting\free\model_save\cather\best_EPE_epoch_20.pth
(42, 21, 2)
(49, 21, 2)


KeyboardInterrupt: 